## 0. Import Dependencies and ros-node

In [ ]:
# %config IPCompleter.use_jedi=False
# !pip install tensorflow==1.15.0 tensorflow-gpu==1.15.0

In [ ]:
import gym
import jaco_gym
import random
import numpy as np 
import rospy

In [ ]:
# You need to first launch Jaco in Gazebo with
# roslaunch kinova_gazebo robot_launch_render.launch kinova_robotType:=j2n6s300     # enable graphic rendering
# roslaunch kinova_gazebo robot_launch_noRender_noSphere.launch kinova_robotType:=j2n6s300   # disable graphic rendering
# such that the code bleow is able to run successfuly.

rospy.init_node("kinova_client", anonymous=True, log_level=rospy.INFO)

## 1. Test Random Environment

In [ ]:
# Environment name:

# JacoReachGazebo-v0 (Jaco arm in Gazebo with Topics)
# JacoReachGazebo-v1 (Jaco arm in Gazebo with actionlib)
# JacoReachGazebo-v2 (Jaco arm in Gazebo with actionlib + fixed target + only 1 joint moving + angle change)
# JacoReachReal-v0 (Physical Jaco arm env)

In [ ]:
env = gym.make('JacoGazebo-v1')
# gym.make?
# ~/gripper_ws/src/motionPlanning/gym/gym/envs/registration.py

In [ ]:
from stable_baselines3.common.env_checker import check_env

In [ ]:
## It will check your custom environment and output additional warnings if needed
from stable_baselines.common.env_checker import check_env
print("starting check")
check_env(env, warn=True)
print("check done")

In [ ]:
print('Action space:')
print(env.action_space)
print(env.action_space.high)
print(env.action_space.low)

In [ ]:
print('State space:')
print(env.observation_space)
print(env.observation_space.high)
print(env.observation_space.low)

In [ ]:
# obs = env.reset()
# action = env.action_space.sample()
# print('random action:', action)
# obs, reward, done, info = env.step(actio9n)

render_flag=True

for episode in range(3):

    obs = env.reset()
    rewards = []

    for t in range(5):

        action = env.action_space.sample()
        obs, reward, done, info = env.step(action)

        print("timestep:", t)
        print("action: ", action)
        print("observation: ", obs)
        print("reward: ", reward)
        print("done: ", done)
        print("info: ", info)

        if done:
            rewards.append(reward)
            break

    print("Episode: {}, Cumulated reward: {}".format(episode, sum(rewards)))
    print("******************")

env.close()

## 2. Build and Train the SB3-Model

In [ ]:
# !pip install stable-baselines3[extra]

In [ ]:
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv

In [ ]:
env_name = 'JacoGazebo-v1'
env = gym.make(env_name)
env = DummyVecEnv([lambda: env])

In [ ]:
import os

log_path = os.path.join('Training', 'Logs')
# model = PPO('MlpPolicy', env, verbose = 1)
model = PPO("MlpPolicy", env, verbose=1, tensorboard_log=log_path)

In [ ]:
# model.learn(total_timesteps=1000)
# model.learn(total_timesteps=10000)
model.learn(total_timesteps=20000)

In [ ]:
# 41 times

In [ ]:
# n_updates = total_timesteps // self.n_batch
# 500/64

In [ ]:
# PPO?
#     n_steps:int=2048,
#     batch_size:int=64,
#     n_epochs:int=10

## 3. Save and Reload Model

In [ ]:
PPO_path = os.path.join('Training', 'Saved Models', 'PPO_model')

In [ ]:
model.save(PPO_path)

In [ ]:
del model

In [ ]:
model = PPO.load(PPO_path, env=env)

## 4. Evaluation

In [ ]:
from stable_baselines3.common.evaluation import evaluate_policy

In [ ]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

In [ ]:
env.close()

In [ ]:
# evaluate_policy?

## 5. Test Model

In [ ]:
# before testing: graphically move the sphere target, if training, comment this line
# self.robot.move_sphere(self.target_vect)
# file name: jaco_gazebo_action_env.py

In [ ]:
obs = env.reset()
while True:
    action, _states = model.predict(obs)
    obs, rewards, done, info = env.step(action)
    env.render()
    if done: 
        print('info', info)
        break

In [ ]:
env.close()

In [ ]:
# # Enjoy trained agent
# obs = env.reset()
# while True:
#     action, _states = model.predict(obs)
#     obs, rewards, dones, info = env.step(action)
# env.close()

## 6. Viewing Logs in Tensorboard

In [ ]:
# log_path = os.path.join('Training', 'Logs')
training_log_path = os.path.join(log_path, 'PPO_1')

In [ ]:
!tensorboard --logdir={training_log_path}

## 7. Adding a callback to the training Stage

In [ ]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold
import os

In [ ]:
save_path = os.path.join('Training', 'Saved Models')
log_path = os.path.join('Training', 'Logs')

In [ ]:
env = gym.make(env_name)
env = DummyVecEnv([lambda: env])

In [ ]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=190, verbose=1)
eval_callback = EvalCallback(env, 
                             callback_on_new_best=stop_callback, 
                             eval_freq=10000, 
                             best_model_save_path=save_path, 
                             verbose=1)

In [ ]:
model = PPO('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)

In [ ]:
model.learn(total_timesteps=20000, callback=eval_callback)

In [ ]:
model_path = os.path.join('Training', 'Saved Models', 'best_model')
model = PPO.load(model_path, env=env)

In [ ]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

In [ ]:
env.close()

## 8. Changing Policies

In [ ]:
net_arch=[dict(pi=[128, 128, 128, 128], vf=[128, 128, 128, 128])]

In [ ]:
model = PPO('MlpPolicy', env, verbose = 1, policy_kwargs={'net_arch': net_arch})

In [ ]:
model.learn(total_timesteps=20000, callback=eval_callback)

## 10. Using an Alternate Algorithm

In [ ]:
from stable_baselines3 import DQN

In [ ]:
model = DQN('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)

In [ ]:
model.learn(total_timesteps=20000, callback=eval_callback)

In [ ]:
dqn_path = os.path.join('Training', 'Saved Models', 'DQN_model')

In [ ]:
model.save(dqn_path)

In [ ]:
model = DQN.load(dqn_path, env=env)

In [ ]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

In [ ]:
env.close()